# data source: <a href="https://qpublic.schneidercorp.com/Application.aspx?AppID=1027&LayerID=21667&PageTypeID=2&PageID=9280" style="color: #ff9d00; text-decoration: none; font-weight: 600;">Forsyth County Qpublic</a>

# 1) Compile home sales data

In [ ]:
import os # allow Python to slither through our file tree
import glob # this module will allow use of wildcard characters to find and combine data
import pandas as pd # pandas dataframes will hold our data

In [ ]:
# navigate to where the CSVs live 
path = 'Data/Glob/' 

# create list of all CSV in the variable 'path' above 
path_files = glob.glob(os.path.join(path, "*.csv"))

# concatenate, remove duplicates
df = pd.concat((pd.read_csv(f) for f in path_files)).drop_duplicates().reset_index(drop=True)

# Describe the dataframe 
print(f'The combined dataframe has {df.shape[0]:,} rows & {df.shape[1]} columns.')

In [ ]:
# Now, show the dataframe
df.head()

# 2) Clean

In [ ]:
# remove unwanted columns
df1 = df.drop(columns=[
    'Parcel ID',
    'Owner Name',
    'Owner Address',
    'Grantor',
    'Grantee',
    'Qualified Sales',
    'Sales Validity',
    'Acres',
    'Parcel  Class ',
    'Tax District',
    'Neighborhood',
    'Zoning'
])

# create the "full address" column for Google Maps to read
df1['full_address'] = df1['Address'].str.title() + ' Forsyth County GA'
df1.head(10)

# set the 'space filler', a URL-encoded stand in for a space character
space_filler = '%20'

# inject the 'space filler' into 'full_address' column
df1['full_address'] = df1['full_address'].str.replace(' ', space_filler)
pd.set_option('display.max_colwidth', 1000)   # display more of the columns
df1.head(10)

# creates the URL column for geocoding
df1['url'] = ['https://www.google.com/maps/search/' + i for i in df1['full_address']]
df1.head()

# 3) Geocode

In [ ]:
from time import sleep # pause the geocoder between "hits"
from tqdm import tqdm # give a status bar
from selenium import webdriver # automate browsing
from selenium.webdriver.chrome.options import Options # run in the background
import re # use regex to strip out lat / long values
import warnings # suppress SettingWithCopyWarning

In [ ]:
# set selenium options
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)

# this empty list will hold our results
results = []

# demo a small subset for the geocoder
df1 = df1.head(25)

# look at the first URL in the dataframe
df1['url'][0]

In [ ]:
# for loop which will iterate over each row and grab the resulting URL
for ind in tqdm(df1.index, colour='#2171b5', desc='Geocoding Progress'):
    try:
        driver.get(df1['url'][ind])
        sleep(3.7)
        url = driver.current_url
        results.append(url)
    except:
        results.append('error')

print('Geocoding complete!')

In [ ]:
# initialize 2 empty lists
lats = []
longs = []

# parse & split the 'results' list 
for item in range(len(results)):
    try:
        found = re.search('/@(.+?),17z', results[item]).group(1)
        lats.append(found.split(',')[0])
        longs.append(found.split(',')[1])
    except:
        lats.append('error')
        longs.append('error')
        
# now add the parsed & cleaned lat/longs as additional columns to our dataframe
df1.loc[:, 'lat'] = lats
df1.loc[:,'long'] = longs

# remove unwanted columns
df1.drop(columns=[
    'full_address',
    'url'
], inplace=True)

# view the results
df1

# 4) Spatial join

In [ ]:
# get 5 years' worth of Forsyth data
forsyth_sales = pd.read_csv('Data/Final/FullSet.csv')
forsyth_sales.drop(columns=[
    'Unnamed: 0'
], inplace=True)

# Look at the table data
forsyth_sales

In [ ]:
import geopandas as gpd # like pandas, but for spatial data

# convert the pandas dataframe to a geopandas geodataframe
forsyth_sales = gpd.GeoDataFrame(
    forsyth_sales, 
    geometry=gpd.points_from_xy(
        forsyth_sales.long, 
        forsyth_sales.lat
    ), 
    crs="EPSG:4326"
)

# visualize the geocoded home sales
forsyth_sales.explore(tiles='CartoDB Positron')

In [ ]:
# Statewide census tracts from Open Data & Mapping Hub
url = "https://services1.arcgis.com/Ug5xGQbHsD8zuZzM/arcgis/rest/services/ACS_2021_Population/FeatureServer/21/query?where=1%3D1&outFields=*&outSR=4326&f=json"
forsyth_tracts = gpd.read_file(url)

# only grab Forsyth County
forsyth_tracts = forsyth_tracts[forsyth_tracts['GEOID'].str.startswith('13117')]

# we're only intersted in the geometry & tract ID
forsyth_tracts = forsyth_tracts[['GEOID', 'geometry']]

# take a look at the data (map):
forsyth_tracts.explore(tiles='CartoDB Positron')

In [ ]:
# create a new Geopandas Geodataframe of spatially joined data
joined_sales = forsyth_sales.sjoin(forsyth_tracts, predicate="within")

# drop other joined field
joined_sales = joined_sales.drop(columns='index_right')

# take a look at the joined data (map)
joined_sales.explore()

# # take a look at the joined dat (table)
# joined_sales.head()

In [ ]:
# Convert our joined_sales geodataframe to a geometry-less pandas dataframe
joined_sales = joined_sales.drop(columns='geometry')

# Export to a CSV to be used in our 'app.py' file
joined_sales.to_csv('Data/Final/FullSet_joined.csv')

print('export complete 🥳')